In [6]:
import torch
from torch.utils.data import Dataset,DataLoader
from glob import glob
from sklearn.model_selection import train_test_split
import ipdb
import rasterio
from PIL import Image
import numpy as np
from tqdm.auto import tqdm
from model import WrapperModel
from dataset import SARDataset
import segmentation_models_pytorch as smp
import torch.nn.functional as F
import pathlib
import os
from torch import nn
import albumentations as A
import ttach as tta
import matplotlib.pyplot as plt

In [2]:
unet_pp=smp.create_model(arch='unetplusplus',classes=1,in_channels=6,encoder_name='tu-maxvit_large_tf_512', encoder_weights=None)
model1=WrapperModel.load_from_checkpoint('/home/syo/work/2024_IEEE_GRSS/src/weights/maxvit-800e/epoch=791-step=72864.ckpt',model=unet_pp,mode='test',map_location=torch.device("cuda"))
model1.eval()

WrapperModel(
  (model): UnetPlusPlus(
    (encoder): TimmUniversalEncoder(
      (model): FeatureListNet(
        (stem): Stem(
          (conv1): Conv2dSame(6, 128, kernel_size=(3, 3), stride=(2, 2))
          (norm1): BatchNormAct2d(
            128, eps=0.001, momentum=0.1, affine=True, track_running_stats=True
            (drop): Identity()
            (act): GELUTanh()
          )
          (conv2): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
        )
        (stages_0): MaxxVitStage(
          (blocks): Sequential(
            (0): MaxxVitBlock(
              (conv): MbConvBlock(
                (shortcut): Downsample2d(
                  (pool): AvgPool2dSame(kernel_size=(2, 2), stride=(2, 2), padding=(0, 0))
                  (expand): Identity()
                )
                (pre_norm): BatchNormAct2d(
                  128, eps=0.001, momentum=0.1, affine=True, track_running_stats=True
                  (drop): Identity()
                  (act):

In [3]:
def _sar_normalization(sar_data):
    """
        normalize the data to (0, 1).

        Band 1: SAR image, VV
        Band 2: SAR image, VH
        Band 3: Merit DEM
        Band 4: Copernicus DEM
        Band 5: ESA World Cover Map
        Band 6: Water occurrence probability

        # 1: 0-32765        x/32765
        # 2:                x/32765
        # 3: -9999:9999     (x+9999)/(9999*2)
        # 4: 0-100          x/100
        # 5: 0-255          x/255
    """
    sar_data=sar_data.astype(np.float32)
    sar_data[0]=sar_data[0]/32765
    sar_data[1]=sar_data[1]/32765
    sar_data[2]=(sar_data[2]+9999)/(9999*2)
    sar_data[3]=(sar_data[3]+9999)/(9999*2)
    sar_data[4]=sar_data[4]/100
    sar_data[5]=sar_data[5]/255

    return sar_data

In [27]:
image_root=pathlib.Path('/home/syo/work/2024_IEEE_GRSS/dataset/Track1/val/images/')
image_list = sorted(list(image_root.glob('*')))
thred=1e-3

num_rows = len(image_list) // 3 + 1
num_cols = 3

plt.figure(figsize=(num_rows,num_cols))

for n,i in tqdm(enumerate(image_list, 1), total=len(image_list)):
    sar_data = rasterio.open(i).read()
    sar_data=_sar_normalization(sar_data=sar_data)
    with torch.no_grad():
            sar_data=torch.tensor(sar_data,dtype=torch.float32).unsqueeze(0).cuda()
            outputs1=model1.forward(sar_data)
            outputs1 = F.sigmoid(outputs1)
            outputs=outputs1
            preds=outputs.detach().cpu().numpy().squeeze(0)[0]
            binary_map = (preds > thred).astype(np.uint8)
            #plt.imshow(preds)
            plt.subplot(num_rows,num_cols,n)
            plt.imshow(binary_map)
plt.show()

  0%|          | 0/349 [00:00<?, ?it/s]/home/syo/opt/mambaforge/envs/kaggle/lib/python3.11/site-packages/rasterio/__init__.py:304: NotGeoreferencedWarning: Dataset has no geotransform, gcps, or rpcs. The identity matrix will be returned.
  dataset = DatasetReader(path, driver=driver, sharing=sharing, **kwargs)
100%|██████████| 349/349 [00:28<00:00, 12.31it/s]
